# Creating Redshift Cluster using the AWS python SDK

## STEP 1: Setup


In [ ]:
# Add imports
import pandas as pd
import boto3
import json
import psycopg2

In [ ]:
# Create administrator IAM role on AWS (manually)

"""
- Create a new IAM user in your AWS account
- Give it `AdministratorAccess`, from `Attach existing policies directly` Tab
- Create access key
- Take note of the access key and secret 
- Edit the file `dwh_ias.cfg` in the same folder as this notebook and fill

KEY=YOUR_AWS_KEY
SECRET=YOUR_AWS_SECRET
"""

In [ ]:
# Load DWH parameters from dwh_ias.cfg
import configparser

config = configparser.ConfigParser()
config.read_file(open("dwh_ias.cfg"))

KEY = config.get("AWS", "KEY")
SECRET = config.get("AWS", "SECRET")

DWH_CLUSTER_TYPE = config.get("DWH", "DWH_CLUSTER_TYPE")
DWH_NUM_NODES = config.get("DWH", "DWH_NUM_NODES")
DWH_NODE_TYPE = config.get("DWH", "DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH", "DWH_CLUSTER_IDENTIFIER")
DWH_DB = config.get("DWH", "DWH_DB")
DWH_DB_USER = config.get("DWH", "DWH_DB_USER")
DWH_DB_PASSWORD = config.get("DWH", "DWH_DB_PASSWORD")
DWH_PORT = config.get("DWH", "DWH_PORT")

DWH_IAM_ROLE_NAME = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame(
    {
        "Param": [
            "DWH_CLUSTER_TYPE",
            "DWH_NUM_NODES",
            "DWH_NODE_TYPE",
            "DWH_CLUSTER_IDENTIFIER",
            "DWH_DB",
            "DWH_DB_USER",
            "DWH_DB_PASSWORD",
            "DWH_PORT",
            "DWH_IAM_ROLE_NAME",
        ],
        "Value": [
            DWH_CLUSTER_TYPE,
            DWH_NUM_NODES,
            DWH_NODE_TYPE,
            DWH_CLUSTER_IDENTIFIER,
            DWH_DB,
            DWH_DB_USER,
            DWH_DB_PASSWORD,
            DWH_PORT,
            DWH_IAM_ROLE_NAME,
        ],
    }
)


In [ ]:
# Create clients for EC2, S3, IAM, and Redshift
region = "us-east-1"

ec2 = boto3.resource(
    "ec2", region_name=region, aws_access_key_id=KEY, aws_secret_access_key=SECRET
)

s3 = boto3.resource(
    "s3", region_name=region, aws_access_key_id=KEY, aws_secret_access_key=SECRET
)

iam = boto3.client(
    "iam", region_name=region, aws_access_key_id=KEY, aws_secret_access_key=SECRET
)

redshift = boto3.client(
    "redshift", region_name=region, aws_access_key_id=KEY, aws_secret_access_key=SECRET
)

## STEP 2: Create IAM role that allows Redshift to access S3 bucket (ReadOnly)

In [ ]:
# Create the IAM role
try:
    print("Creating a new IAM Role")
    dwhRole = iam.create_role(
        Path="/",
        RoleName=DWH_IAM_ROLE_NAME,
        Description="Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
            {
                "Statement": [
                    {
                        "Action": "sts:AssumeRole",
                        "Effect": "Allow",
                        "Principal": {"Service": "redshift.amazonaws.com"},
                    }
                ],
                "Version": "2012-10-17",
            }
        ),
    )
    print(f"Successfully created a new IAM Role: {DWH_IAM_ROLE_NAME}")
except Exception as e:
    print(e)

In [ ]:
print("Attaching AmazonS3ReadOnlyAccess Policy")
response_s3 = iam.attach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess"
)["ResponseMetadata"]["HTTPStatusCode"]

if response_s3 == 200:
    print("Successfully attached AmazonS3ReadOnlyAccess policy to the IAM role")

print("Attaching AmazonRedshiftFullAccess Policy")
response_redshift = iam.attach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/AmazonRedshiftFullAccess"
)["ResponseMetadata"]["HTTPStatusCode"]

if response_redshift == 200:
    print("Successfully attached AmazonRedshiftFullAccess policy to the IAM role")

In [ ]:
# Get and print the IAM role ARN
print("Get the IAM role ARN")
roleArn = iam.get_role(RoleName=DWH_IAM_ROLE_NAME)["Role"]["Arn"]

print(roleArn)

## STEP 3:  Create Redshift Cluster

In [ ]:
# Create Redshift cluster subnet group
ec2_client = boto3.client(
    "ec2",
    region_name=region,
    aws_access_key_id=KEY,
    aws_secret_access_key=SECRET,
)
response = ec2_client.describe_vpcs(Filters=[{"Name": "isDefault", "Values": ["true"]}])
default_vpc_id = response["Vpcs"][0]["VpcId"]

subnet_response = ec2_client.describe_subnets(
    Filters=[{"Name": "vpc-id", "Values": [default_vpc_id]}]
)
subnet_ids = [subnet["SubnetId"] for subnet in subnet_response["Subnets"]]
subnet_group_name = "default-vpc-redshift-subnet-group"
description = "Subnet group for Redshift in default VPC"

try:
    response = redshift.create_cluster_subnet_group(
        ClusterSubnetGroupName=subnet_group_name,
        Description=description,
        SubnetIds=subnet_ids,
    )
    print(f"Cluster Subnet Group {subnet_group_name} created successfully.")
except Exception as e:
    print(f"Error creating cluster subnet group: {e}")

In [ ]:
# Create Redshift cluster
try:
    response = redshift.create_cluster(
        # TODO: add parameters for hardware
        ClusterType=DWH_CLUSTER_TYPE,
        NodeType=DWH_NODE_TYPE,
        NumberOfNodes=int(DWH_NUM_NODES),
        # TODO: add parameters for identifiers & credentials
        DBName=DWH_DB,
        ClusterIdentifier=DWH_CLUSTER_IDENTIFIER,
        MasterUsername=DWH_DB_USER,
        MasterUserPassword=DWH_DB_PASSWORD,
        # TODO: add parameter for role (to allow s3 access)
        IamRoles=[roleArn],
        ClusterSubnetGroupName=subnet_group_name,
    )
    print(f"Redshift Cluster {DWH_CLUSTER_IDENTIFIER} created successfully.")
except Exception as e:
    print(e)

In [ ]:
# Get and print the Redshift cluster identifier
def prettyRedshiftProps(props):
    pd.set_option("display.max_colwidth", None)
    keysToShow = [
        "ClusterIdentifier",
        "NodeType",
        "ClusterStatus",
        "MasterUsername",
        "DBName",
        "Endpoint",
        "NumberOfNodes",
        "VpcId",
    ]
    x = [(k, v) for k, v in props.items() if k in keysToShow]
    return pd.DataFrame(data=x, columns=["Key", "Value"])


myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)[
    "Clusters"
][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# Save the DWH endpoint and IAM role ARN
DWH_ENDPOINT = myClusterProps["Endpoint"]["Address"]
DWH_ROLE_ARN = myClusterProps["IamRoles"][0]["IamRoleArn"]

config["DWH"]["DWH_ENDPOINT"] = DWH_ENDPOINT
config["DWH"]["DWH_ROLE_ARN"] = DWH_ROLE_ARN

with open("dwh_ias.cfg", "w") as configfile:
    config.write(configfile)

print("DWH_ENDPOINT :: ", DWH_ENDPOINT)
print("DWH_ROLE_ARN :: ", DWH_ROLE_ARN)

## STEP 4: Open an incoming  TCP port to access the cluster endpoint

In [ ]:
# Authorize default security group to allow traffic from anywhere to the Redshift port
try:
    vpc = ec2.Vpc(id=myClusterProps["VpcId"])
    defaultSg = list(vpc.security_groups.all())[0]
    print(defaultSg)
    defaultSg.authorize_ingress(
        GroupName=defaultSg.group_name,
        CidrIp="0.0.0.0/0",
        IpProtocol="TCP",
        FromPort=int(DWH_PORT),
        ToPort=int(DWH_PORT),
    )
except Exception as e:
    print(e)

## STEP 5: Connect to the cluster

In [ ]:
# Check Redshift cluster connection
conn_string = (
    f"postgresql://{DWH_DB_USER}:{DWH_DB_PASSWORD}@{DWH_ENDPOINT}:{DWH_PORT}/{DWH_DB}"
)
print(conn_string)
try:
    conn = psycopg2.connect(conn_string)
    print("Connection successful")
    conn.close()
except Exception as e:
    print(f"Connection failed: {e}")

## STEP 6: Clean up your resources

<b><font color='red'>DO NOT RUN THIS UNLESS YOU ARE SURE <br/> 
    We will be using these resources in the next exercises</span></b>

In [ ]:
# Delete Redshift cluster

redshift.delete_cluster(
    ClusterIdentifier=DWH_CLUSTER_IDENTIFIER, SkipFinalClusterSnapshot=True
)

In [ ]:
# Check Redshift cluster deletion
myClusterProps = redshift.describe_clusters(ClusterIdentifier=DWH_CLUSTER_IDENTIFIER)[
    "Clusters"
][0]
prettyRedshiftProps(myClusterProps)

In [ ]:
# Delete IAM role
iam.detach_role_policy(
    RoleName=DWH_IAM_ROLE_NAME,
    PolicyArn="arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess",
)
iam.delete_role(RoleName=DWH_IAM_ROLE_NAME)